<a href="https://colab.research.google.com/github/sznajdr/sumpdf/blob/main/sumpdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title packages
!pip install Sentencepiece
!pip install ipywidgets
!pip install pdfplumber transformers
import nltk
nltk.download('punkt')
from IPython.display import clear_output
clear_output(wait=True)
print("Everything installed.")

Everything installed.


In [28]:
#@title upload
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  with open('file.pdf', 'wb') as f:
    f.write(uploaded[filename])
    from IPython.display import clear_output
    clear_output(wait=True)
print("File uploaded successfully!")

File uploaded successfully!


In [29]:
#@title seiten auswählen
import pdfplumber

filename = 'file.pdf'
start_page = int(input("Enter the start page number: "))
end_page = int(input("Enter the end page number: "))
output_filename = input("Enter the output filename: ")

if not output_filename.endswith('.txt'):
    output_filename += '.txt'

with pdfplumber.open(filename) as pdf:
    all_pages_text = ''
    for i, page in enumerate(pdf.pages):
        if i+1 >= start_page and i+1 <= end_page:
            extracted_text = page.extract_text()
            all_pages_text += extracted_text
            print(f"Extracting text from page {i+1}...")
            from IPython.display import clear_output
            clear_output(wait=True)
    with open(output_filename, 'w') as f:
        f.write(all_pages_text)
    print("Text extraction complete.")

Text extraction complete.


In [ ]:
#@title sum alle
import os
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

for filename in os.listdir(os.getcwd()):
    if filename.endswith('.txt'):
        with open(filename, 'r') as f:
            text = f.read()

        inputs = tokenizer([text], truncation=True, return_tensors='pt')

        # Generate Summary
        summary_ids = model.generate(inputs['input_ids'], num_beams=10, early_stopping=True, min_length=600, max_length=10024)
        summarized_text = ([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids])

        # Save Summary
        output_filename = f"sum_{filename}"
        with open(output_filename, 'w') as f:
            f.write(summarized_text[0])
        print(f"Summary of {filename} saved to {output_filename}")


Summary of 15_35.txt saved to sum_15_35.txt


In [10]:
#@title sum auswählen
import os
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')
from IPython.display import clear_output
clear_output(wait=True)

# Prompt the user to enter the filename of a file in the current working directory
filename = input("Enter the filename of the file in the current working directory: ")

# Check if the file has a .txt extension
if not filename.endswith('.txt'):
    print("Invalid file extension. Please enter the filename of a text file.")
else:
    # Check if the file exists in the current working directory
    if not os.path.isfile(filename):
        print("File does not exist in the current working directory.")
    else:
        # Read the text from the file
        with open(filename, 'r') as f:
            text = f.read()

        inputs = tokenizer([text], truncation=True, return_tensors='pt')

        # Generate Summary
        summary_ids = model.generate(inputs['input_ids'], num_beams=33, early_stopping=True, min_length=1400, max_length=8524)
        summarized_text = ([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids])

        # Save Summary
        output_filename = f"sum_{filename}"
        with open(output_filename, 'w') as f:
            f.write(summarized_text[0])
        print(f"Summary of {filename} saved to {output_filename}")


Enter the filename of the file in the current working directory: alle.txt
Summary of alle.txt saved to sum_alle.txt


In [34]:
#@title ***` - ⏏ `*** NLP Lex Rank Summarizer

import requests
import os
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def summarize_file(filename, length):
    with open(filename, encoding='utf-8') as file:
        text = file.read()
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, length)
    return summary

current_directory = os.getcwd()
text_files = [f for f in os.listdir(current_directory) if f.endswith(".txt")]

# create new folders for output and original files
if not os.path.exists("sums"):
    os.makedirs("sums")
if not os.path.exists("origs"):
    os.makedirs("origs")

for file in text_files:
    file_path = os.path.join(current_directory, file)
    summary = summarize_file(file_path, 12)
    output_filename = f"sum_{file}"
    output_path = os.path.join("sums", output_filename)
    with open(output_path, 'w') as output_file:
        for sentence in summary:
            output_file.write(f"{sentence}\n")
    # move original file to "origs" folder
    orig_path = os.path.join("origs", file)
    os.rename(file_path, orig_path)

print("Summary complete.")


Summary complete.
